# Implementación de *Simulated Annealing* para el problema del agente viajero

**Importar las librerías necesarias para poder trabajar con los datos y poner un 'seed'**

In [ ]:
import pandas as pd
import numpy as np
from numpyencoder import NumpyEncoder
import matplotlib.pyplot as plt
import os
import json

np.random.seed(626)

**Leer los datos (la matriz de distancias) y crear un DataFrame a partir de ellos**

In [ ]:
CITIES = np.empty((128, 3))

with open('coordinates.txt', 'r') as f:
    for index, line in enumerate(f.readlines()):
        line = line.strip().split()
        CITIES[index] = [index, float(line[0]), float(line[1])]

DATA = pd.read_csv('cities.csv', header=None)

# Vamos a imprimir los primeros 5 para asegurarnos de que todo está bien
DATA.head(5)

**Hacer un primer camino aleatorio**

In [ ]:
PATH = np.random.permutation(CITIES)

**Definir la función del costo del camino actual**

In [ ]:
def cost(path: np.array) -> float:
    """Calcula el costo de la ruta elegida al momento

    Args:
        path (np.array): Array que contiene la ruta que se debe de seguir al momento

    Returns:
        float: El costo de la ruta
    """
    global DATA
    
    cost = 0
    
    # Calcular el costo del camino
    for i in range(len(path) - 1):
        cost += DATA.iloc[int(path[i, 0]), int(path[i + 1, 0])]
        
    # Sumar el costo de volver al principio
    cost += DATA.iloc[int(path[-1, 0]), int(path[0, 0])]
    return cost

**Definir la función de obtener un nuevo camino 'vecino' del actual**

In [ ]:
def generate_path(path: np.array) -> np.array:
    """Genera una ruta tomando dos elementos aleatorios de la ruta, y haciendo un 'reverse' de todos los elementos 
        dentro del array formado por estas dos ciudades como extremos

    Args:
        path (np.array): La ruta actual

    Returns:
        np.array: La nueva ruta
    """
    # Tomar dos elementos aleatorios
    i, j = np.random.choice([i for i in range(0, 128)], size=2, replace=False)
    
    # Hacer el swap
    path[i : j] = path[i : j][::-1]
    
    return path

## Función que usa todas las funciones anteriores para generar la mejor ruta

In [ ]:
def simulated_annealing(path: np.array, t: float, l: int, max_loops: int = 2000000, t_update: float = 0.999) -> list:
    """Función que implementa el algoritmo de Simulated Annealing

    Args:
        path (np.array): La ruta actual
        t (float): Temperatura
        l (int): Cada cuantas iteraciones se debe de actualizar la temperatura
        max_loops (int, optional): Cantidad máxima de iteraciones. Defaults to 1000000.
        t_update (float, optional): Factor de actualización de la temperatura. Defaults to 0.99.

    Returns:
        np.array: La nueva ruta
    """
    diff = 0
    k = 0
    cost_path = cost(path)
    results = []
    t_values = []
    information = {
        'T': t,
        'L': l,
        't_update': t_update,
        'max_loops': max_loops,
        'paths': [
            path
        ]
    }

    while k < max_loops:
        # Generar una nueva ruta
        new_path = generate_path(path)

        # Calcular la diferencia de costos
        cost_new_path = cost(new_path)
        diff = cost_new_path - cost_path

        # Si la diferencia es menor que 0, entonces la nueva ruta es mejor, por lo que la actual se debe de actualizar
        if diff < 0:
            path = new_path
            cost_path = cost_new_path

        # Si la diferencia es mayor que 0, entonces la nueva ruta es peor, por lo que se debe de probar si debe de ser aceptada
        elif np.random.rand() < np.exp(-diff / t):
            path = new_path
            cost_path = cost_new_path

        # Escribir el costo dentro del array de resultados y T para graficar
        results.append(cost_path)
        t_values.append(t)

        # Aumentar K y cada L pasos disminuir T
        k += 1
        if k % l == 0:
            t *= t_update
            
        # Agregar la nueva ruta a la información
        information['paths'].append(path)
            
    # Guardar la información dentro de un archivo
    with open(f'files/SA_t={t}_l={l}_tupdate={t_update}_d={results[-1]}.json', 'w') as file:
        json.dump(information, file, indent=4, sort_keys=True,separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
    
    return results, t_values
    
    

**Ejecutar el programa y graficar el costo para T desde 1000 hasta 9000 (brincando 1000)**

In [ ]:
execute = True

if execute:
    for T in np.arange(1000, 10000, 1000):
        l = 100
        t_update = 0.999
        max_loops = 200000
        
        # run algorithm
        results, t_values = simulated_annealing(path=PATH, t=T, l=l, max_loops=max_loops, t_update=t_update)
    
        # graph results
        plt.figure(figsize=(6,4), facecolor='w')
        plt.plot(range(len(results)), results)
        plt.title(f'Cost of Simulated Annealing with T={T}')
        plt.xlabel('k iterations')
        plt.ylabel('Cost')
        
        # save plot on disk
        plt.savefig(f'./plots/SA_t={T}_l={l}_tupdate={t_update}_d={results[-1]}.png')
        print(f"saved ./plots/SA_t={T}_l={l}_tupdate={t_update}_d={results[-1]}.png")
        plt.clf()
        
        ########################
        # graph results
        plt.plot(range(len(t_values)), t_values)
        plt.title(f'Value of T')
        plt.xlabel('k iterations')
        plt.ylabel('T')
        
        # save plot on disk
        plt.savefig(f'./plots/T_t={T}_l={l}_tupdate={t_update}_maxloops={max_loops}.png')
        plt.clf()

Nos damos cuenta después de todas las corridas que con T=3000 es el mejor resultado: ```135,434``` km.

Es por eso que vamos a usar esos datos ```(T=3000, L=100, t_update=0.997)``` para generar nuestra animación del agente viajero

<img src="plots/SA_t=3000_l=100_tupdate=0.997_d=135434.png"/>

A continuación podemos ver la gráfica de como va disminuyendo T a través del tiempo

<img src="plots/T_t=3000_l=100_tupdate=0.997_maxloops=100000.png"/>


**Primero vamos a graficar todas las coordenadas**

In [ ]:
COORDS = np.empty((128, 2))

with open('coordinates.txt', 'r') as f:
    for index, line in enumerate(f.readlines()):
        line = line.strip().split()
        COORDS[index] = [float(line[0]), float(line[1])]

In [ ]:
plt.figure(figsize=(8,6), facecolor='w')

plt.plot(COORDS.T[0], COORDS.T[1], 'o', c='black')
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Cities")

plt.savefig(f'./map.png')
plt.show()